# Práctica 2: Caso práctico con Mongo DB.

In [ ]:
# Imports
import pymongo
import json
from datetime import datetime
from pprint import pprint

# **EJERCICIO 2**

# *Apartado A*

Completa los datos de conexión a la BBDD MongoDB.

### Consulta en Python

In [ ]:
# Conexiones con MongoDB
db_name = "practica_2_mongodb"
db_uri = "mongodb://localhost:27017"
db_client = pymongo.MongoClient(db_uri)
db = db_client[db_name]

In [ ]:
ruta_coleccion_usuarios = r"usuarios.json"
ruta_coleccion_partidas = r"partidas.json"

In [5]:
# Colecciones
nombre_coleccion_usuarios = "usuarios"
nombre_coleccion_partidas = "partidas"

coleccion_usuarios = db[nombre_coleccion_usuarios]
coleccion_partidas = db[nombre_coleccion_partidas]

### Explicación

# *Apartado B*

* Genera una función (“check_if_exist(db, nombre_coleccion))” que compruebe si las colecciones existen y, en caso de existir, las elimine. 
* A su vez, si la colección se ha borrado, que escriba un mensaje por pantalla y, si no existe, también.  
* Usa dicha función con nuestras dos colecciones.

### Consulta en Python

In [ ]:
def check_if_exist(db, nombre_coleccion):

    if nombre_coleccion in db.list_collection_names():
        db[nombre_coleccion].drop()
        print(f"La colección '{nombre_coleccion}' ha sido borrada.")
    else:
        print(f"La colección '{nombre_coleccion}' no existe.")

# Comprobar si existen
check_if_exist(db, nombre_coleccion_usuarios)
check_if_exist(db, nombre_coleccion_partidas)

### Explicación

# *Apartado C*

* Carga los ficheros de datos JSON proporcionados en sus colecciones correspondientes.  
* Para ello, genera una función genérica (“load_data(coleccion, ruta_fichero_json))” y úsala.

### Consulta en Python

In [ ]:
def load_data(coleccion, ruta_fichero_json):
    with open(ruta_fichero_json, 'r') as file:
        data = json.load(file)
    coleccion.insert_many(data)
    print(f"Se ha insertado la informacion en la coleccion '{coleccion}'")

load_data(coleccion_usuarios, ruta_coleccion_usuarios)
load_data(coleccion_partidas, ruta_coleccion_partidas)

### Explicación

# *Apartado D*

* Obtén las puntuaciones más altas de cada jugador, es decir, la puntuación máxima de cada uno.
* Genera un ranking ordenado de mayor a menor.

### Consulta en Python

In [ ]:
pipeline = [
    { "$group": { "_id": "$user_id", "maxScore": { "$max": "$score" } } },
    { "$project": {"_id": 0, "username": "$_id", "maxScore": 1}},
    { "$sort": { "maxScore": -1 } }
]

documentos = coleccion_partidas.aggregate(pipeline)

for doc in documentos:
    print(doc)


### Explicación

# *Apartado E*

* Lista todos los jugadores que hayan superado el nivel 3.
* Ordénalos de mayor a menor por nivel.

### Consulta en Python

In [ ]:
documentos = coleccion_partidas.find(
        { "level": { "$gt": 3 } }
    ).sort("level", -1)

for doc in documentos:
    pprint(doc)

### Explicación

# *Apartado F*

* Lista todas las partidas jugadas entre el 01/03/2024 y el 30/04/2024.

### Consulta en Python

In [ ]:
fecha_inicio = "2024-03-01"
fecha_fin = "2024-03-31"

documentos = coleccion_partidas.find({ 
    "date": { 
        "$gte": fecha_inicio, 
        "$lte": fecha_fin 
    } 
})

for doc in documentos:
    pprint(doc)


### Explicación

# *Apartado G*

* Genera un ranking de jugadores por puntuación total acumulada.
* Ordénalo de mayor a menor.

### Consulta en Python

In [ ]:
pipeline = [
    { "$group": { "_id": "$user_id", "totalScore": { "$sum": "$score" } } },
    { "$sort": { "totalScore": -1 } }
]

documentos = coleccion_partidas.aggregate(pipeline)

for doc in documentos:
    print(doc)

### Explicación

# *Apartado H*

* Muestra al usuario con la partida más larga en duración

### Consulta en Python

In [ ]:
# OPCION 1

pipeline = [
    { "$group": {"_id": "$user_id", "maxDuration": {"$max": "$duration"} } },
    { "$project": {"_id": 0, "username": "$_id", "maxDuration": 1} },
    {"$sort": {"maxDuration": -1}},
    {"$limit": 1}
]

documentos = coleccion_partidas.aggregate(pipeline)

for doc in documentos:
    pprint(doc)

In [ ]:
# OPCION 2

documentos = coleccion_partidas.find({}, { "username": "$user_id", "duration": 1, "_id": 0 }).sort("duration", -1).limit(1)

for doc in documentos:
    pprint(doc)

### Explicación

# *Apartado I*

* Muestra el username, correo electrónico y puntuación del jugador con la puntuación más alta.  
* La salida debe contener solo el registro de dicho jugador.

### Consulta en Python

In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "partidas",
            "localField": "username",
            "foreignField": "user_id",
            "as": "user_games"
        }
    },
    { "$unwind": "$user_games" },
    { 
        "$group": { 
            "_id": "$username", 
            "username": { "$first": "$username" }, 
            "email": { "$first": "$email" }, 
            "maxScore": { "$max": "$user_games.score" } 
        } 
    },
    { "$sort": { "maxScore": -1 } },
    { 
        "$project": { 
            "_id": 0, 
            "username": 1, 
            "email": 1, 
            "maxScore": 1 
        } 
    },
    { "$limit": 1 }
]

documentos = coleccion_usuarios.aggregate(pipeline)

for doc in documentos:
    pprint(doc)


### Explicación

* En esta etapa hacemos la join entre la coleccion de usuarios y partidas.
```
{
    "$lookup": {
        "from": "partidas",
        "localField": "username",
        "foreignField": "user_id",
        "as": "user_games"
    }
}
```
* Desacoplamos la lista de documentos resultantes en un documento separado para cada join de usuarios con partidas.
```
{ "$unwind": "$user_games" }
```
* En esta etapa agrupamos por username y calculamos el maxScore.
* Como quemos conservar el username y el email para la siguiente etapa, debemos seleccionar la primera ocurrencia de cada uno de ellos porque si no MongoDB no sabe como agregarlos y da error.
```
"$group": { 
    "_id": "$username", 
    "username": { "$first": "$username" }, 
    "email": { "$first": "$email" }, 
    "maxScore": { "$max": "$user_games.score" } 
} 
```
* Ordenamos de forma descendente.
```
{ "$sort": { "maxScore": -1 } }
```
* Mostramos los campos username, email y maxScore. Quitamos de la vista el valor "_id" mostrado por defecto.
```
{ 
    "$project": { 
        "_id": 0, 
        "username": 1, 
        "email": 1, 
        "maxScore": 1 
    } 
},
```
* Limitamos la salida a un único documento
```
{ "$limit": 1 }
```